---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd
import numpy as np
import re

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)

df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
def date_sorter():
    
    import pandas as pd
    import numpy as np
    import re

    dates = ["•04/20/2009;", "04/20/09;", "4/20/09;", "4/3/09;",
    "•Mar-20-2009;", "Mar 20, 2009;", "March 20, 2009;", "Mar. 20, 2009;", "Mar 20 2009;", "October 14 1974",
    "•20 Mar 2009;", "20 March 2009;", "20 Mar. 2009;", "20 March, 2009","2June, 1999",
    "•Mar 20th, 2009;", "Mar 21st, 2009;", "Mar 22nd, 2009",
    "•Feb 2009;", "Sep 2009;", "Oct 2010",
    "•6/2008;", "12/2009",
    "•2009;", "2010"] # rango con la tipología de las fechas existentes en el .txt

    dates = pd.Series(dates)
    
    # "•04/20/2009;", "04/20/09;", "4/20/09;", "4/3/09;"
    rule_1 = r'(0?\d|1[0-2])[\/\-](0?\d|[12]\d|30|31)[\/\-](\d{4}|\d{2})'
    df1 = df.str.extractall(rule_1) # con .extractall hacemos un df con todas las líneas del .txt con una tipología de fecha = rule_1
    df1.columns = ['month','day','year'] # adecuada para la tipología que estamos tratando
    df1=df1.reset_index()
    
    
    # "•Mar-20-2009;", "Mar 20, 2009;", "March 20, 2009;", "Mar. 20, 2009;", "Mar 20 2009;", "October 14 1974",
    # Mar 20th, 2009;", "Mar 21st, 2009;", "Mar 22nd, 2009"
    rule_2 = r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Ago|Sep|Oct|Nov|Dec)[a-z\.]*[ -](\d{1,2})[a-z\.\,]*[ -](\d{4})'
    df2= df.str.extractall(rule_2)
    df2.columns = ['month','day','year']
    df2 = df2.reset_index()
    
    # "•20 Mar 2009;", "20 March 2009;", "20 Mar. 2009;", "20 March, 2009","2June, 1999"
    # "•Feb 2009;", "Sep 2009;", "Oct 2010"
    rule_3 = r'(\d{1,2})?[ -]?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z\.\,]*[ -](\d{4})'
    df3= df.str.extractall(rule_3)
    df3.columns = ['day','month','year']
    df3 = df3.reset_index()
    
    # "•6/2008;", "12/2009",
    rule_4 =r'(\d{1,2})[/](\d{4})'
    df4 = df.str.extractall(rule_4)
    df4.insert(0,column='day',value=np.nan) # al no tener días, tenemos que crear una columna de 0 (o de NaN) para respetar el formato
    df4.columns = ['day','month','year']
    df4 = df4.reset_index()
    
    rule_5 = r'(\d{4})'
    df5 = df.str.extractall(rule_5)
    df5.insert(0, column='day', value=np.nan) #al no tener días, tenemos que crear una columna de 0 (o de NaN) para respetar el formato
    df5.insert(1, column='month', value=np.nan) # al no tener meses, tenemos que crear una columna de 0 (o de NaN) para respetar el formato
    df5.columns = ["month", "day", "year"]
    df5=df5.reset_index()
    
    output = df1.append(df2[~df2.level_0.isin(df1.level_0)]) # df1 añade las filas de df2 cuyo level_0 no exista en df1
    output = output.append(df3[~df3.level_0.isin(output.level_0)]) # df1+df2 añade las filas de df3 cuyo level_0 no exista en df1+df2
    output = output.append(df4[~df4.level_0.isin(output.level_0)]) # df1+df2+df3 añade las filas de df4 cuyo level_0 no exista en df1+df2+df3
    output = output.append(df5[~df5.level_0.isin(output.level_0)]) # df1+df2+df3+df4 añade las filas de df5 cuyo level_0 no exista en df1+df2+df3+df4
    
    output = pd.DataFrame(output,columns = ["level_0", "match", "day", "month","year"]) # reconfiguramos el df de output
    output['year'] = np.where(output.year.apply(len)==2,"19"+output.year,output.year) # reformateamos la columna año, adecuando los 2 digitos a 4 digitos
    output = output.fillna("1") # y añadimos un 1 a los NaN. Así, cuando no haya ni día ni mes será 1 de enero
    
    # Falta formatear los meses. Cambiar Enero, febrero, etc, por números
    
    rang = []
    for i in range(1,13):
        rang.append(i)

    ranges = dict(zip(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'],rang))
    
    output.month = output.month.replace(ranges) # reemplazamos cada mes por su ordinal
    
    #Transformamos todas las filas de str a int para poder ordenar
    output['month'] = output.month.astype(int)
    output['day'] = output.day.astype(int)
    output['year'] = output.year.astype(int)
    
    # Y creamos una columna formato datetime para cada fila
    
    output["date"] = pd.to_datetime(output.loc[:,["year", "month", "day"]])
    output = output.sort_values(["date", "level_0"]).reset_index(drop=True) # Ordenamos por fecha
    
    
    return output.level_0,output.date

date_sorter()

C:\Users\aleex\AppData\Local\Temp/ipykernel_24736/772952018.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = df1.append(df2[~df2.level_0.isin(df1.level_0)]) # df1 añade las filas de df2 cuyo level_0 no exista en df1
C:\Users\aleex\AppData\Local\Temp/ipykernel_24736/772952018.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(df3[~df3.level_0.isin(output.level_0)]) # df1+df2 añade las filas de df3 cuyo level_0 no exista en df1+df2
C:\Users\aleex\AppData\Local\Temp/ipykernel_24736/772952018.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(df4[~df4.level_0.isin(output.level_0)]) # df1+df2+df3 añade las filas de df4 cuyo level_0 no exista en df1+df2+df3
C:\Users\a

(0        9
 1       84
 2        2
 3       53
 4       28
       ... 
 495    427
 496    141
 497    186
 498    161
 499    413
 Name: level_0, Length: 500, dtype: int64,
 0     1971-04-10
 1     1971-05-18
 2     1971-07-08
 3     1971-07-11
 4     1971-09-12
          ...    
 495   2016-05-01
 496   2016-05-30
 497   2016-10-13
 498   2016-10-19
 499   2016-11-01
 Name: date, Length: 500, dtype: datetime64[ns])